# Quality Check

In [ ]:
#import functions
import plotly.express as px
from pathlib import Path
import numpy as np
import skimage.io
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
from util import pil_imread
import tifffile as tf

def read_images(files, num_channels=4, max_project = True):

    #Read in images
    imgs = []
    for i in tqdm(range(len(files))):
        try:
            img = pil_imread(files[i], num_channels = num_channels, swapaxes=True)
            if img.shape[1] != num_channels:
                img = pil_imread(files[i], num_channels = num_channels, swapaxes=False)
        except:
            img = pil_imread(files[i], num_channels=None, swapaxes=True)
            if img.shape[1] != num_channels:
                img = pil_imread(files[i], num_channels=None, swapaxes=False)
        if max_project == True:
            imgs.append(np.max(img,axis=0))
        else:
            imgs.append(img)
            
    return imgs

In [ ]:
# Load the images: 
input_directory = "/segmentation/image/path/*.tif"
files=glob.glob(input_directory)
#organize files numerically
key = [int(re.search('MMStack_Pos(\\d+)', f).group(1)) for f in files]
files = list(np.array(files)[np.argsort(key)])

In [ ]:
#read in images
#don't forget to adjust number of channels if it is not 4!
#max project if you have many slices but still single cell
imgs = read_images(files, num_channels=2, max_project=True)

In [ ]:
#Z, C, X, Y 
imgs[0].shape

In [ ]:
#write out max projected or stacked masks in same directory
import tifffile as tf
for i, img in enumerate(imgs):
    name = files[i].split("/")[-1]
    tf.imwrite(name,img)

# Load masks

In [ ]:
# Load segmentation mask
seg_mask = np.load('./images/MMStack_Pos0.ome_seg.npy', allow_pickle=True)

In [ ]:
#check quality
plt.imshow(seg_mask.item()["outlines"], cmap="binary", vmin=0, vmax=0.1)
plt.show()

In [ ]:
from glob import glob
import os

#grab all segmentatio files from cellpose3
files = glob("/path/to/cellpose/outputs/mask_generation/images/*.npy")

#create directroy
parent = Path(files[0]).parent
while "output" not in os.listdir(parent):
    parent = parent.parent
direct = parent / "output" / "masks"
direct.mkdir(exist_ok = True, parents=True)

#number of z copies if you are using the same mask for different z spot locations
z_copies = 9

for src in files:
    #create file name
    name = src.split("/")[-1]
    name2 = name.replace("ome_seg.npy", ".tif")

    #load masks from stacked array
    seg_mask = np.load(src, allow_pickle=True)
    mask = seg_mask.item()["masks"]
    if z_copies > 0:
        for z in range(z_copies):
            name3 = name.replace(".ome_seg.npy", f"_z{z}.tif")
            #write out masks
            output_path = direct / name3
            tf.imwrite(str(output_path), mask)
    else:
        #write out masks
        output_path = direct / name2
        tf.imwrite(str(output_path), mask)